## '股票大池435_ths.csv'布林带日内交易策略_回测2022-06-04
### 设置操作标的：context.stockpool = [ '股票大池435_ths.csv' ]

In [ ]:
# from tick_trade_api import TradeAPI
# # 输入指定账户
# trade_api=TradeAPI('79855043',order_policy=MarketPolicy)


In [ ]:
# import pandas as pd
# import numpy as np
# import talib

In [1]:
source_code="""

# 初始化函数
def init(context):
    #导入模型
    import pandas as pd
    import numpy as np
    import talib
    #设置Boll模型参数
    context.timeperiod = 20 
    context.nbdevup = 2 
    context.nbdevdn = 2    
    # 运作周期
    context.runstep = 3
    # 设置基准收益
    #set_benchmark('000300.SH')
    #set_benchmark(002060.SZ')
    context.N = 1200  #分钟线_波动差均值系数
    context.N300 = '000300.SH'
    # 设置操作标的：
    #context.stockpool = ['002625.SZ', '600657.SH', '002060.SZ', '600359.SH']
    #context.stockpool = ['002060.SZ']
    file_path = '股票大池435_ths.csv'
    context.stockpool = []
    context.stockpool_1 = []
    context.holdstock0 = []
    df =pd.read_csv(file_path)
    context.stockpool_1 = list(df)[1:]
    print('len(context.stockpool_1)=', len(context.stockpool_1))
    context.n = 30 # 最大持股数
    #运用I问财进行股票筛选
    get_iwencai('未停牌,上市时间超过2年')
    
    # 设置手续费
    set_commission(PerShare(type='stock',cost=0.00025))
    #设置可变滑点1%，表示买入价为实际价格乘101%，卖出价为实际价格乘99%
    set_slippage(PriceSlippage(0.01))
    # 固定盈利
    context.win = 5 #盈利几个点卖出
    # 单笔买入仓位
    context.buypct = 0.01
    # 止盈
    context.stopwin = 100
    # 止损
    context.stoploss = 0.07  

    # 设置初识持仓
    #set_holding_stocks(dict(zip(context.stockpool,[702200]*len(context.stockpool))))
    set_holding_stocks(dict(zip(context.stockpool,[0]*len(context.stockpool))))
    
    # 记录日内买入日期时间
    context.stocktrade = {}
    for stock in context.stockpool:
        context.stocktrade[stock] = []
    
    # 获取当前持仓
    holdstock = list(context.portfolio.stock_account.positions.keys())
    print('持仓股票holdstock：', holdstock) 
    # 查询持仓情况
    market_value = context.portfolio.stock_account.market_value
    total_value = context.portfolio.stock_account.total_value
    stocklist = list(context.portfolio.stock_account.positions)
    print('当前持仓市值:', market_value, '持仓股票stocklist:', stocklist, '账户总资产:', int(total_value))
    

# 每个交易日开市前，打印出目前的账户信息.
def before_trading(context):
    g.stocks=context.iwencai_securities    
    holdstock1 = list(context.portfolio.stock_account.positions.keys())
    stock_t = []
    for security in g.stocks:
        if security in context.stockpool_1:
            stock_t.append(security)
    context.stockpool = list(set(stock_t))[::]
    print('len(context.stockpool)=', len(context.stockpool))
    #log.info(context.portfolio)
    
    
    holdstock1 = list(context.portfolio.stock_account.positions.keys())
    stock_t = []
    for security in g.stocks:
        if security in holdstock1:
            stock_t.append(security)
    context.holdstock0 = list(set(stock_t))[::]
    
    
    
    # 记录日内买入日期时间
    context.stocktrade = {}
    for stock in context.stockpool:
        context.stocktrade[stock] = []    
    
    
    
def handle_bar(context, bar_dict):    
    # 导入Python-talib库
    import talib
    import numpy as np
    import pandas as pd
    # 获取当前分钟
    date = get_datetime().strftime('%Y-%m-%d %H:%M:%S')
    time1 = get_datetime().strftime('%M')
    time = get_datetime().strftime('%M')

    # 一、判断盈亏条件
    # 1.1 止盈止损
    # 获取当前持仓
    holdstock2 = list(context.portfolio.stock_account.positions.keys())
    stock_t = []
    for security in context.holdstock0:
        if security in holdstock2:
            stock_t.append(security)
    holdstock = list(set(stock_t))[::]
    
    for stock in holdstock:   # 循环持仓股票
        # 导入Python-talib库
        import talib
        #import numpy as np
        #import pandas as pd
        
        # 获取成本价
        cost_basis = context.portfolio.positions[stock].cost_basis
        # 获取数据
        df = history(stock, ['close','high','low'], 2400,'1m', True, 'pre', is_panel=1)
        # 提取数据
        close_price = df['close']
        high_price = df['high']
        low_price = df['low']
        new_price = df['close'].iloc[-1] #最新收盘价
        
        # 获取开、高、低价格
        data = history(stock, ['open','high','low'], context.N+1, '1m')
        # 计算上行波动
        dif_up = data['high']/data['open']-1
        # 计算下行波动
        dif_down = 1-data['low']/data['open']
        # 计算波动差值
        dif_T = dif_up-dif_down
        # 计算波动差值均值
        dif_ma = pd.rolling_mean(dif_T,context.N)
        # 若均值为正，则买入或持仓
        #if dif_ma.values[-1] > 0 and value==0:
        # 若均值为负，则卖出或空仓
        #if dif_ma.values[-1] < 0 and value> 0:

        
        # 获取可卖股数等持仓情况
        amount = context.portfolio.positions[stock].amount
        available_amount = context.portfolio.positions[stock].available_amount
        
        # 计算指标，获取BOLL值。布林带日内交易卖出策略
        upperband, middleband, lowerband = talib.BBANDS(close_price.values, timeperiod=context.timeperiod, nbdevup=context.nbdevup, nbdevdn=context.nbdevdn)
        T_value = upperband-lowerband
        T_value = T_value[19:]
        mean = np.mean(T_value)
        new = T_value[-1]
        #if (available_amount>0 and T_value[-2] >T_value[-1] and new<mean and new_price>upperband[-1]) or dif_ma.values[-1]<0:
        if (available_amount>0 and T_value[-2] >T_value[-1] and new<mean and new_price>upperband[-1]):
            print(stock,'当前处于相对高点{}'.format(new_price))
            #order_percent(stock, -context.buypct)
            order(stock, -available_amount)
            # 记录这次卖出时间
            context.stocktrade[stock].append(date)
            market_value = context.portfolio.stock_account.market_value
            total_value = context.portfolio.stock_account.total_value
            print(stock,"波动向下或布林相对高点卖出:{},卖出价格:{},毛盈利{}%".format(stock,new_price,int(10000*(new_price/cost_basis -1))/100))
            print('==>卖出股票:', stock, available_amount, '股, 持仓股票市值:', market_value ,'账户总资产:', int(total_value))
            # 移除本股票本轮交易时间记录
            context.stocktrade[stock] = []
            #print('记录这次卖出后时间应该为==>', context.stocktrade[stock])
            break
        
        
        if new_price/cost_basis-1>context.stopwin and available_amount>0:
            # 卖出所有股票
            print('持仓股数:', amount, '可卖股数:', available_amount, "止盈卖出 %s" % (stock), available_amount, '股')
            order(stock,-available_amount)
            # 移除本股票本轮交易时间记录
            context.stocktrade[stock] = []
            market_value = context.portfolio.stock_account.market_value
            total_value = context.portfolio.stock_account.total_value
            print('==>持仓市值', int(market_value), '账户总资产:', int(total_value))
            break

        elif new_price/cost_basis-1< -context.stoploss and available_amount>0:
            # 卖出该股全部
            print('持仓股数:', amount, '可卖股数:', available_amount, "止盈卖出 %s" % (stock), available_amount, '股')
            order(stock,-available_amount)
            # 移除本股票本轮交易时间记录
            context.stocktrade[stock] = [] 
            market_value = context.portfolio.stock_account.market_value
            total_value = context.portfolio.stock_account.total_value
            stocklist = list(context.portfolio.stock_account.positions)
            print('==>持仓市值', int(market_value), '账户总资产:', int(total_value))
            break
            
        # 1.2 达到获利目标后回落卖出 
        # 取交易时间数据
        if len(context.stocktrade[stock])>0:
            d1 = context.stocktrade[stock][0]
            dd1 = d1[:4]+d1[5:7]+d1[8:10]+' '+d1[11:16]
            dd2 = date
            df = get_price(stock, dd1, dd2,'1m', ['close', 'open', 'low', 'high'], skip_paused = False, fq = 'pre', is_panel = 1)

            # 计算本轮首次买入后至最新价期间的最高价及最大涨幅%
            max_price = df['high'].max()
            max_zf = (max_price-cost_basis)/cost_basis*100
            # 当最大涨幅大于context.win时候
            if max_zf>context.win and max_zf<20:
                # 计算本轮首次买入后至最新价期间的冲高回落幅度%
                hlfd = ((max_price-new_price)/(max_price-cost_basis))*100
                if hlfd > 20 and available_amount>0:
                    # 卖出该股全部
                    print('持仓股数:', amount, '可卖股数:', available_amount, "回落卖出:{},卖出价格:{},毛盈利{}%".format(stock,new_price,int(10000*(new_price/cost_basis -1))/100))
                    order(stock,-available_amount)
                    # 移除本股票本轮交易时间记录
                    context.stocktrade[stock] = []
                    market_value = context.portfolio.stock_account.market_value
                    total_value = context.portfolio.stock_account.total_value
                    print('==>持仓市值', int(market_value), '账户总资产:', int(total_value))
                    break
                    
            # 当最大涨幅大于等于20%时候
            if max_zf>=20:
                # 计算本轮首次买入后至最新价期间的冲高回落幅度%
                hlfd = ((max_price-new_price)/(max_price-cost_basis))*100
                if hlfd > 10 and available_amount>0:
                    # 卖出该股全部
                    print('持仓股数:', amount, '可卖股数:', available_amount, "回落卖出:{},卖出价格:{},毛盈利{}%".format(stock,new_price,int(10000*(new_price/cost_basis -1))/100))
                    order(stock,-available_amount)
                    # 移除本股票本轮交易时间记录
                    context.stocktrade[stock] = []
                    market_value = context.portfolio.stock_account.market_value
                    total_value = context.portfolio.stock_account.total_value
                    stocklist = list(context.portfolio.stock_account.positions)
                    print('==>持仓市值', int(market_value), '账户总资产:', int(total_value))
                    break

    
    # 二、布林带日内交易买入策略+波动向上策略
    # 000300.SZ的小时线MA24趋势向上+收盘价在该MA24上方才运行买入策略
    # 获取数据
    dfN = history(context.N300, ['close'], 1465,'1m', True, 'pre', is_panel=1)
    close_price_N = int(dfN['close'].iloc[-1]*100)/100
    dfN_MA24_a = int(dfN['close'][24:-1].mean()*1000)/1000
    dfN_MA24_b = int(dfN['close'][:-25].mean()*1000)/1000
    if dfN_MA24_b<dfN_MA24_a:
        #print('dfN_MA24_a/b=', dfN_MA24_a, dfN_MA24_b, 'close_price_N=',close_price_N)
        # 循环目标股票池股票
        for stock in context.stockpool:
            # 导入Python-talib库
            #import talib
            #import numpy as np
            #import pandas as pd

            # 获取数据
            df1 = history(stock, ['close','high','low'], 2400,'1m', True, 'pre', is_panel=1)
            df_d = history(stock, ['open'], 2,'1d', True, 'pre', is_panel=1)
            open_price = df_d['open'].iloc[-1]
            # 提取数据
            close_price = df1['close']
            high_price = df1['high']
            low_price = df1['low']
            new_price = df1['close'].iloc[-1]

            # 获取开、高、低价格
            data = history(stock, ['open','high','low'], context.N+1, '1m')
            # 计算上行波动
            dif_up = data['high']/data['open']-1
            # 计算下行波动
            dif_down = 1-data['low']/data['open']
            # 计算波动差值
            dif_T = dif_up-dif_down
            # 计算波动差值均值
            dif_ma = pd.rolling_mean(dif_T,context.N)
            # 若均值为正，则买入或持仓
            #if dif_ma.values[-1] > 0 and value==0:
            # 若均值为负，则卖出或空仓
            #if dif_ma.values[-1] < 0 and value> 0:

            # 计算指标，获取BOLL值
            available_amount = context.portfolio.positions[stock].available_amount
            upperband, middleband, lowerband = talib.BBANDS(close_price.values, timeperiod=context.timeperiod, nbdevup=context.nbdevup, nbdevdn=context.nbdevdn)
            T_value = upperband-lowerband
            T_value = T_value[19:]
            mean = np.mean(T_value)
            new = T_value[-1]
            # 最新价的涨幅
            new_zf = int((new_price - open_price)/open_price * 10000)/100

            if T_value[-2] >T_value[-1] and new<mean and new_price<lowerband[-1] and new_zf<3 and dif_ma.values[-1]>0:            
                order_percent(stock, context.buypct)
                market_value = context.portfolio.stock_account.market_value
                total_value = context.portfolio.stock_account.total_value            
                # 记录这次买入时间
                context.stocktrade[stock].append(date)            
                print(stock, '涨幅{}当前处于相对低点{}及波动向上'.format(new_zf,new_price),'买入股票', '持仓股票市值', market_value , '账户总资产:', int(total_value))
                print('context.stocktrade[stock].append(date),记录这次买入时间==>', context.stocktrade[stock])
            

            
# 收盘后运行函数,用于储存自定义参数、全局变量,执行盘后选股等
def after_trading(context):
    
    log.info('一天结束')            

"""

# # 系统重置
# try:
#     import shutil
#     shutil.rmtree('persist/研究环境策略')
# except:
#     pass

# # 回测运行
# # research_strategy(source_code, start_date=None, end_date=None, capital_base=100000, frequency='DAILY', stock_market='STOCK', benchmark=None)
# research_strategy(source_code, start_date='20191230', end_date='20220527', capital_base=float(10000000), frequency='MINUTE', stock_market='STOCK', benchmark=None)




In [ ]:
#     #context.N = 55 #日线
#     context.N = 1200  #分钟线
# #设置买卖条件，每个交易频率（日/分钟/tick）调用一次   
# def handle_bar(context,bar_dict):
#     # 获取开、高、低价格
#     #data = history(context.security, ['open','high','low'], context.N+1, '1d')
#     data = history(context.security, ['open','high','low'], context.N+1, '1m')
#     # 计算上行波动
#     dif_up = data['high']/data['open']-1
#     # 计算下行波动
#     dif_down = 1-data['low']/data['open']
#     # 计算波动差值
#     dif_T = dif_up-dif_down
#     # 计算波动差值均值
#     dif_ma = pd.rolling_mean(dif_T,context.N)
#     # 持股市值
#     value = context.portfolio.stock_account.market_value
#     # 若均值为正，则买入或持仓
#     if dif_ma.values[-1] > 0 and value==0:
#         #order_target_percent(context.security,1)
#     # 若均值为负，则卖出或空仓
#     if dif_ma.values[-1] < 0 and value> 0:
#         #order_target(context.security,0)

In [2]:
# 系统重置
try:
    import shutil
    shutil.rmtree('persist/研究环境策略')
except:
    pass


In [ ]:
# 模拟运行
#rtrade = research_trade('研究环境策略', source_code, capital_base=float(10000000), frequency='MINUTE', trade_api=trade_api)#trade_api可传入List[TradeAPI]，实现多账户交易



In [ ]:
# 回测运行
# research_strategy(source_code, start_date=None, end_date=None, capital_base=100000, frequency='DAILY', stock_market='STOCK', benchmark=None)
#research_strategy(source_code, start_date='20191230', end_date='20220527', capital_base=float(4551000), frequency='MINUTE', stock_market='STOCK', benchmark=None)


In [3]:
# 回测运行
# research_strategy(source_code, start_date=None, end_date=None, capital_base=100000, frequency='DAILY', stock_market='STOCK', benchmark=None)
research_strategy(source_code, start_date='20180901', end_date='20190301', capital_base=float(10000000), frequency='MINUTE', stock_market='STOCK', benchmark=None)


2018-09-03 00:00:00.000000 - INFO - len(context.stockpool_1)= 435
2018-09-03 00:00:00.000000 - INFO - 持仓股票holdstock： []
2018-09-03 00:00:00.000000 - INFO - 当前持仓市值: 0 持仓股票stocklist: [] 账户总资产: 10000000
2018-09-03 09:00:00.000000 - INFO - len(context.stockpool)= 278
2018-09-03 09:31:00.000000 - INFO - 600763.SH 涨幅-1.85当前处于相对低点53.98及波动向上 买入股票 持仓股票市值 97164.0 账户总资产: 9999489
2018-09-03 09:31:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2018-09-03 09:31:00']
2018-09-03 09:31:00.000000 - INFO - 600039.SH 涨幅-0.83当前处于相对低点3.55及波动向上 买入股票 持仓股票市值 196564.0 账户总资产: 9998967
2018-09-03 09:31:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2018-09-03 09:31:00']
2018-09-03 09:33:00.000000 - INFO - 600875.SH 涨幅-0.97当前处于相对低点7.13及波动向上 买入股票 持仓股票市值 295707.0 账户总资产: 9998483
2018-09-03 09:33:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2018-09-03 09:33:00']
2018-09-03 09:58:00.000000 - INFO - 600107.SH 涨幅-2.53当前处于相对低点5.39及波动向上 买入股票 持仓股票市值

2018-09-18 14:29:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2018-09-18 14:29:00']
2018-09-18 14:35:00.000000 - INFO - 600298.SH 涨幅-0.87当前处于相对低点25.91及波动向上 买入股票 持仓股票市值 197788.0 账户总资产: 9984863
2018-09-18 14:35:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2018-09-18 14:35:00']
2018-09-18 14:41:00.000000 - INFO - 600667.SH 涨幅-0.96当前处于相对低点6.16及波动向上 买入股票 持仓股票市值 297096.0 账户总资产: 9984151
2018-09-18 14:41:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2018-09-18 14:41:00']
2018-09-18 14:49:00.000000 - INFO - 600416.SH 涨幅0.28当前处于相对低点7.1及波动向上 买入股票 持仓股票市值 395985.0 账户总资产: 9983692
2018-09-18 14:49:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2018-09-18 14:49:00']
2018-09-18 15:30:00.000000 - INFO - 一天结束
2018-09-19 09:00:00.000000 - INFO - len(context.stockpool)= 278
2018-09-19 09:34:00.000000 - INFO - 600400.SH 涨幅2.79当前处于相对低点4.04及波动向上 买入股票 持仓股票市值 496899.0 账户总资产: 9985106
2018-09-19 09:34:00.00000

2018-09-20 09:31:00.000000 - INFO - 300069.SZ 波动向下或布林相对高点卖出:300069.SZ,卖出价格:11.24,毛盈利-0.87%
2018-09-20 09:31:00.000000 - INFO - ==>卖出股票: 300069.SZ 8800 股, 持仓股票市值: 3476888.0 账户总资产: 9978076
2018-09-20 09:31:00.000000 - INFO - 000887.SZ 涨幅0.66当前处于相对低点10.55及波动向上 买入股票 持仓股票市值 3576058.0 账户总资产: 9977555
2018-09-20 09:31:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2018-09-20 09:31:00']
2018-09-20 09:32:00.000000 - INFO - 600107.SH 当前处于相对高点5.74
2018-09-20 09:32:00.000000 - INFO - 600107.SH 波动向下或布林相对高点卖出:600107.SH,卖出价格:5.74,毛盈利0.7%
2018-09-20 09:32:00.000000 - INFO - ==>卖出股票: 600107.SH 17500 股, 持仓股票市值: 3473712.0 账户总资产: 9974858
2018-09-20 09:33:00.000000 - INFO - 601012.SH 当前处于相对高点12.67
2018-09-20 09:33:00.000000 - INFO - 601012.SH 波动向下或布林相对高点卖出:601012.SH,卖出价格:12.67,毛盈利-0.79%
2018-09-20 09:33:00.000000 - INFO - ==>卖出股票: 601012.SH 15600 股, 持仓股票市值: 3278006.0 账户总资产: 9975725
2018-09-20 09:34:00.000000 - INFO - 600021.SH 当前处于相对高点7.65
2018-09-20 09:34:00.000000 - INFO - 600021.

2018-09-20 11:09:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2018-09-20 10:27:00', '2018-09-20 11:09:00']
2018-09-20 11:14:00.000000 - INFO - 600400.SH 当前处于相对高点4.04
2018-09-20 11:14:00.000000 - INFO - 600400.SH 波动向下或布林相对高点卖出:600400.SH,卖出价格:4.04,毛盈利-0.35%
2018-09-20 11:14:00.000000 - INFO - ==>卖出股票: 600400.SH 73800 股, 持仓股票市值: 5023518.0 账户总资产: 9943174
2018-09-20 11:14:00.000000 - INFO - 300122.SZ 涨幅0.72当前处于相对低点43.21及波动向上 买入股票 持仓股票市值 5118580.0 账户总资产: 9942697
2018-09-20 11:14:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2018-09-20 10:28:00', '2018-09-20 10:55:00', '2018-09-20 11:14:00']
2018-09-20 11:15:00.000000 - INFO - 600617.SH 涨幅0.17当前处于相对低点5.85及波动向上 买入股票 持仓股票市值 5218105.0 账户总资产: 9942837
2018-09-20 11:15:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2018-09-20 11:15:00']
2018-09-20 11:19:00.000000 - INFO - 002062.SZ 涨幅-0.47当前处于相对低点4.2及波动向上 买入股票 持仓股票市值 5312303.0 账户总资产: 9937817
2018-09-20 11:19:00.000000 - I

2018-09-20 13:27:00.000000 - INFO - 600667.SH 当前处于相对高点6.22
2018-09-20 13:27:00.000000 - WARN - 600667.SH 下单股数为0,订单无效.
2018-09-20 13:27:00.000000 - INFO - 600667.SH 波动向下或布林相对高点卖出:600667.SH,卖出价格:6.22,毛盈利-0.64%
2018-09-20 13:27:00.000000 - INFO - ==>卖出股票: 600667.SH 0 股, 持仓股票市值: 6682276.0 账户总资产: 9935828
2018-09-20 13:28:00.000000 - INFO - 600667.SH 当前处于相对高点6.22
2018-09-20 13:28:00.000000 - WARN - 600667.SH 下单股数为0,订单无效.
2018-09-20 13:28:00.000000 - INFO - 600667.SH 波动向下或布林相对高点卖出:600667.SH,卖出价格:6.22,毛盈利-0.64%
2018-09-20 13:28:00.000000 - INFO - ==>卖出股票: 600667.SH 0 股, 持仓股票市值: 6680172.0 账户总资产: 9933724
2018-09-20 13:29:00.000000 - INFO - 600667.SH 当前处于相对高点6.22
2018-09-20 13:29:00.000000 - WARN - 600667.SH 下单股数为0,订单无效.
2018-09-20 13:29:00.000000 - INFO - 600667.SH 波动向下或布林相对高点卖出:600667.SH,卖出价格:6.22,毛盈利-0.64%
2018-09-20 13:29:00.000000 - INFO - ==>卖出股票: 600667.SH 0 股, 持仓股票市值: 6678664.0 账户总资产: 9932216
2018-09-20 13:30:00.000000 - INFO - 600667.SH 当前处于相对高点6.23
2018-09-20 13:30:00.000000 - WARN - 60

...

2019-01-17 14:04:00.000000 - WARN - 600332.SH 下单股数为0,订单无效.
2019-01-17 14:04:00.000000 - INFO - 600332.SH 涨幅0.55当前处于相对低点36.0及波动向上 买入股票 持仓股票市值 7993904.0 账户总资产: 7994214
2019-01-17 14:04:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2019-01-17 14:04:00']
2019-01-17 14:05:00.000000 - INFO - 002475.SZ 当前处于相对高点15.06
2019-01-17 14:05:00.000000 - WARN - 002475.SZ 下单股数为0,订单无效.
2019-01-17 14:05:00.000000 - INFO - 002475.SZ 波动向下或布林相对高点卖出:002475.SZ,卖出价格:15.06,毛盈利-0.87%
2019-01-17 14:05:00.000000 - INFO - ==>卖出股票: 002475.SZ 0 股, 持仓股票市值: 7999237.0 账户总资产: 7999547
2019-01-17 14:06:00.000000 - INFO - 002475.SZ 当前处于相对高点15.06
2019-01-17 14:06:00.000000 - WARN - 002475.SZ 下单股数为0,订单无效.
2019-01-17 14:06:00.000000 - INFO - 002475.SZ 波动向下或布林相对高点卖出:002475.SZ,卖出价格:15.06,毛盈利-0.87%
2019-01-17 14:06:00.000000 - INFO - ==>卖出股票: 002475.SZ 0 股, 持仓股票市值: 8001128.0 账户总资产: 8001438
2019-01-17 14:07:00.000000 - INFO - 002475.SZ 当前处于相对高点15.02
2019-01-17 14:07:00.000000 - WARN - 002475.SZ 下单股数为0,订单无效

2019-01-17 14:28:00.000000 - INFO - ==>卖出股票: 002475.SZ 0 股, 持仓股票市值: 7589868.0 账户总资产: 7974643
2019-01-17 14:29:00.000000 - INFO - 002475.SZ 当前处于相对高点15.06
2019-01-17 14:29:00.000000 - WARN - 002475.SZ 下单股数为0,订单无效.
2019-01-17 14:29:00.000000 - INFO - 002475.SZ 波动向下或布林相对高点卖出:002475.SZ,卖出价格:15.06,毛盈利-0.87%
2019-01-17 14:29:00.000000 - INFO - ==>卖出股票: 002475.SZ 0 股, 持仓股票市值: 7590385.0 账户总资产: 7975160
2019-01-17 14:29:00.000000 - INFO - 000729.SZ 涨幅-1.38当前处于相对低点5.7及波动向上 买入股票 持仓股票市值 7669615.0 账户总资产: 7974605
2019-01-17 14:29:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2019-01-17 14:29:00']
2019-01-17 14:30:00.000000 - INFO - 002475.SZ 当前处于相对高点15.07
2019-01-17 14:30:00.000000 - WARN - 002475.SZ 下单股数为0,订单无效.
2019-01-17 14:30:00.000000 - INFO - 002475.SZ 波动向下或布林相对高点卖出:002475.SZ,卖出价格:15.07,毛盈利-0.81%
2019-01-17 14:30:00.000000 - INFO - ==>卖出股票: 002475.SZ 0 股, 持仓股票市值: 7669242.0 账户总资产: 7974232
2019-01-17 14:31:00.000000 - INFO - 002475.SZ 当前处于相对高点15.07
2019-01-17 14:31:00.000

2019-01-17 14:48:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2019-01-17 14:48:00']
2019-01-17 14:48:00.000000 - WARN - 603060.SH 下单股数为0,订单无效.
2019-01-17 14:48:00.000000 - INFO - 603060.SH 涨幅0.84当前处于相对低点20.2及波动向上 买入股票 持仓股票市值 7939433.0 账户总资产: 7939599
2019-01-17 14:48:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2019-01-17 14:48:00']
2019-01-17 14:49:00.000000 - INFO - 002475.SZ 当前处于相对高点14.96
2019-01-17 14:49:00.000000 - WARN - 002475.SZ 下单股数为0,订单无效.
2019-01-17 14:49:00.000000 - INFO - 002475.SZ 波动向下或布林相对高点卖出:002475.SZ,卖出价格:14.96,毛盈利-1.53%
2019-01-17 14:49:00.000000 - INFO - ==>卖出股票: 002475.SZ 0 股, 持仓股票市值: 7941770.0 账户总资产: 7941936
2019-01-17 14:49:00.000000 - WARN - 603060.SH 下单股数为0,订单无效.
2019-01-17 14:49:00.000000 - INFO - 603060.SH 涨幅0.84当前处于相对低点20.2及波动向上 买入股票 持仓股票市值 7941770.0 账户总资产: 7941936
2019-01-17 14:49:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2019-01-17 14:48:00', '2019-01-17 14:49:00']
2019-01-1

2019-01-18 09:46:00.000000 - INFO - 000400.SZ 当前处于相对高点10.55
2019-01-18 09:46:00.000000 - INFO - 000400.SZ 波动向下或布林相对高点卖出:000400.SZ,卖出价格:10.55,毛盈利-2.46%
2019-01-18 09:46:00.000000 - INFO - ==>卖出股票: 000400.SZ 7400 股, 持仓股票市值: 6819603.0 账户总资产: 7998995
2019-01-18 09:49:00.000000 - INFO - 000014.SZ 当前处于相对高点9.67
2019-01-18 09:49:00.000000 - INFO - 000014.SZ 波动向下或布林相对高点卖出:000014.SZ,卖出价格:9.67,毛盈利-0.72%
2019-01-18 09:49:00.000000 - INFO - ==>卖出股票: 000014.SZ 8200 股, 持仓股票市值: 6743987.0 账户总资产: 8002178
2019-01-18 09:56:00.000000 - INFO - 603098.SH 涨幅-0.44当前处于相对低点13.36及波动向上 买入股票 持仓股票市值 6821512.0 账户总资产: 8000643
2019-01-18 09:56:00.000000 - INFO - context.stocktrade[stock].append(date),记录这次买入时间==> ['2019-01-18 09:56:00']
2019-01-18 10:00:00.000000 - INFO - 002260.SZ 当前处于相对高点3.06
2019-01-18 10:00:00.000000 - INFO - 002260.SZ 波动向下或布林相对高点卖出:002260.SZ,卖出价格:3.06,毛盈利1.45%
2019-01-18 10:00:00.000000 - INFO - ==>卖出股票: 002260.SZ 27000 股, 持仓股票市值: 6750735.0 账户总资产: 8011702
2019-01-18 10:02:00.000000 - INFO - 600084.

CustomException: Traceback (most recent call last):
  File strategy.py, line 266 in handle_bar
    
    --> stock = '600061.SH'
    --> new_zf = 8.87
    --> dif_ma = 2019-01-11 10:30:00         NaN
2019-01-11 10:31:00         NaN
2019-01-11 10:32:00         NaN
2019-01-11 10:33:00         NaN
2019-01-11 10:34:00         NaN
 ...
    --> bar_dict = BarMap()
    --> np = <module 'numpy' from '/opt/conda/lib/python3.5/site-packages/numpy/__init__.py'>
    --> data =                      high   low  open
2019-01-11 10:30:00  3.41  3.40  3.40
2019-01-11 10:31:00  3.42  3.40  3.41
2019-01-11 10:32:00  3.42  3.41  3.42
2019-01- ...
    --> dfN =                          close
2019-01-10 10:06:00  3080.2419
2019-01-10 10:07:00  3081.3586
2019-01-10 10:08:00  3082.8269
2019-01-10 10:09:00  3084.0518
2019- ...
    --> talib = <module 'talib' from '/opt/conda/lib/python3.5/site-packages/talib/__init__.py'>
    --> time1 = '30'
    --> df_d =             open
2019-01-16  9.77
2019-01-17  9.60
   